Import Libraries

In [189]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import math 
import cv2
import pytesseract
import re
from time import time
import time
from decimal import ROUND_UP
from math import ceil
import pandas as pd
import sys, os
import streamlit as st
import selenium

User Inputs

In [161]:
#application_path = os.path.dirname(sys.executable)

URL = input('URL: ')
# userMaxPrice = input('Enter Maximum Price: ')
# priceIterator = input('Price Iterator: ')
# minBedNo = input('Minimum number of bedrooms: ')

userMaxPrice = '700000'
priceIterator = '50000'
minBedNo = '2'

# splits the URL to the relevant area only and ignores all the features selected on the website
beforeURL = URL.split('%')[0] + '%'
afterURL = URL.split('%')[1]
URL = afterURL.split('&')[0]
generatedURL = beforeURL + URL + '&minBedrooms=' + minBedNo
print(generatedURL)

https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E92826&minBedrooms=2


Get all the property addresses

In [162]:
# rightmove main URL
### initialise search ### 

allLinks = np.array([])
allLinksCombined = np.array([])
cleanLinks = np.array([])
minPrice = 50000
maxPrice = minPrice + int(priceIterator)

pageIterator = (int(userMaxPrice)-minPrice)/int(priceIterator)
for x in range(int(pageIterator)):
    URL = generatedURL + '&maxPrice=' + str(maxPrice) + '&minPrice=' + str(minPrice)
    page = requests.get(URL)

    # parse search results card
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id='l-container')

    # index for iterator (remember each page has 24 ads)
    noOfResults = soup.find('div', attrs={'id': 'searchHeader'})
    noOfResults = noOfResults.select('span')[0].text
    noOfResults = re.sub(r',', '', noOfResults)
    noOfResults = ceil(int(noOfResults)/24) # round up to nearest page (note max page no is 42)

    print(noOfResults, " Pages available for min price = ", minPrice, " max price = ", maxPrice)

    i = 0
    all_ids = np.array([])
    for x in range(noOfResults):
        URL = generatedURL + '&maxPrice=' + str(maxPrice) + '&minPrice=' + str(minPrice) + '&sortType=18&index='+ str(i)
        print('Scrapped Page No.', x+1)
        page = requests.get(URL)

        # parse search results card
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id='l-container')

        # Extract and store ids
        links = results.select('a')
        for card in links:
            ids = card.get('id')
            ids = ids.strip() if ids is not None else ''
            all_ids = np.append(all_ids, ids)

        # clean array and add links
        all_ids = all_ids[all_ids != '']
        cleanIds = np.char.strip(all_ids, 'prop')
        allLinks = np.char.add(cleanIds, '#/?channel=RES_BUY')
        allLinks = np.char.add('https://www.rightmove.co.uk/properties/', allLinks)
    
        i = i + 24

    allLinksCombined = np.append(allLinksCombined, allLinks)
    minPrice = minPrice + int(priceIterator) 
    maxPrice = maxPrice + int(priceIterator) 

# removes empty and repeated links 
cleanLinks = np.char.replace(allLinksCombined, 'https://www.rightmove.co.uk/properties/0#/?channel=RES_BUY', '')
cleanLinks = cleanLinks[cleanLinks != '']
cleanLinks = np.unique(cleanLinks)

print('Number of links acquired: ', len(cleanLinks))
print(cleanLinks)


1  Pages available for min price =  50000  max price =  100000
Scrapped Page No. 1
1  Pages available for min price =  100000  max price =  150000
Scrapped Page No. 1
2  Pages available for min price =  150000  max price =  200000
Scrapped Page No. 1
Scrapped Page No. 2
2  Pages available for min price =  200000  max price =  250000
Scrapped Page No. 1
Scrapped Page No. 2
5  Pages available for min price =  250000  max price =  300000
Scrapped Page No. 1
Scrapped Page No. 2
Scrapped Page No. 3
Scrapped Page No. 4
Scrapped Page No. 5
11  Pages available for min price =  300000  max price =  350000
Scrapped Page No. 1
Scrapped Page No. 2
Scrapped Page No. 3
Scrapped Page No. 4
Scrapped Page No. 5
Scrapped Page No. 6
Scrapped Page No. 7
Scrapped Page No. 8
Scrapped Page No. 9
Scrapped Page No. 10
Scrapped Page No. 11
20  Pages available for min price =  350000  max price =  400000
Scrapped Page No. 1
Scrapped Page No. 2
Scrapped Page No. 3
Scrapped Page No. 4
Scrapped Page No. 5
Scrapped 

Fishing Properties

In [163]:
i = 0
counter = 0
all_info = np.array([])
data = np.array([])

for property in cleanLinks:
    start = time()
    ad = requests.get(property)
    soup = BeautifulSoup(ad.content, "html.parser")

    generalInfo = soup.body.find('div', attrs={'_4hBezflLdgDMdftURKTWh'})
    findTenure = int(len(generalInfo.select('dd')))

    # ensures only freehold properties
    if generalInfo.select('dd')[findTenure-1].text.lower() == 'freehold':

        # extract property data
        propertyAddress = soup.select('h1')[0].text
        propertyType = generalInfo.select('dd')[0].text
        noOfBeds = generalInfo.select('dd')[1].text
        priceLink = soup.body.find('div', attrs={'class': '_1gfnqJ3Vtd1z40MlC0MzXu'})
        price = priceLink.select('span')[0].text
        adDateLink = soup.body.find('div', attrs={'class': '_2nk2x6QhNB1UrxdI5KpvaF'})
        date = re.sub('<div class="_2nk2x6QhNB1UrxdI5KpvaF">', '', str(adDateLink))
        date = re.sub('</div>', '', str(date))
        newIds = re.findall(r'\b\d+\b', str(cleanLinks[i])) # use new property id
        mapLocation = np.char.add('https://www.rightmove.co.uk/properties/', str(newIds[0]))
        floorPlan = np.char.add(mapLocation, '#/floorplan?activePlan=1&channel=RES_BUY')
        mapLocation = np.char.add(mapLocation, '#/map?channel=RES_BUY')
        
        # filter through the images to find the floor plan link
        floorPlanImage = requests.get(floorPlan)
        floorPlanImage = BeautifulSoup(floorPlanImage.content, "html.parser")
        floorPlanImage = floorPlanImage.find('div', attrs={'id': 'root'})
        for j in range(len(floorPlanImage.select('img'))):
            nextFloorPlanImage = floorPlanImage.select('img')[j].get_attribute_list('src',str)
            nextFloorPlanImage = ' '.join(nextFloorPlanImage)
            if '_max_296x197' in nextFloorPlanImage:
                foundFloorPlanImage = nextFloorPlanImage.replace('_max_296x197', '')
                break
            else:
                foundFloorPlanImage = 'None'
                pass
        
        i = i+1
        end = time()
        print(str(i)+' out of '+str(len(cleanLinks))+f' It took {round(end - start,2)} seconds!') 

        data = [str(price), str(propertyAddress), str(propertyType), str(noOfBeds), str(foundfloorPlanImage), str(mapLocation), str(date)]
        all_info = np.append(all_info, data)
        counter = counter + 1
        
    else:
        #print('skipped...')
        i = i+1
        end = time()
        print(str(i)+' out of '+str(len(cleanLinks))+f' It took {round(end - start,2)} seconds!') 
        continue
    

# split the array into equally divided arrays
all_info = np.split(all_info, counter)
print(all_info)

1 out of 3698 It took 0.25 seconds!
2 out of 3698 It took 0.28 seconds!
3 out of 3698 It took 0.21 seconds!
4 out of 3698 It took 0.29 seconds!
5 out of 3698 It took 0.22 seconds!
6 out of 3698 It took 0.22 seconds!
7 out of 3698 It took 0.27 seconds!
8 out of 3698 It took 0.55 seconds!
9 out of 3698 It took 0.23 seconds!
10 out of 3698 It took 0.23 seconds!
11 out of 3698 It took 0.25 seconds!
12 out of 3698 It took 0.24 seconds!
13 out of 3698 It took 0.33 seconds!
14 out of 3698 It took 0.56 seconds!
15 out of 3698 It took 0.23 seconds!
16 out of 3698 It took 0.25 seconds!
17 out of 3698 It took 0.23 seconds!
18 out of 3698 It took 0.63 seconds!
19 out of 3698 It took 0.24 seconds!
20 out of 3698 It took 0.21 seconds!
21 out of 3698 It took 0.26 seconds!
22 out of 3698 It took 0.25 seconds!
23 out of 3698 It took 0.22 seconds!
24 out of 3698 It took 0.27 seconds!
25 out of 3698 It took 0.24 seconds!
26 out of 3698 It took 0.42 seconds!
27 out of 3698 It took 0.28 seconds!
28 out of 

Image Processing

In [164]:
import importlib 
from numpy import empty
import searchFunction
importlib.reload(searchFunction) # forces the definition to reload in case you make changes to it
from searchFunction import search 

updated_info = np.array([])

for info in range(len(all_info)):
    print(str(info+1)+' out of '+str(len(all_info)))
    try:
        if not all_info[info][4] == 'None':
            # Load the img
            req = urllib.request.urlopen(all_info[info][4])
            arr = np.asarray(bytearray(req.read()))
            img = cv2.imdecode(arr, -1) # 'Load it as it is'

            # Display loaded image
            # cv2.imshow('Floor Plan', img)
            # if cv2.waitKey() & 0xff == 27: quit()

            # OCR
            pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
            configs = r'--psm 6'
            try:
                rawText = pytesseract.image_to_string(img, config=configs)
            except:
                continue
                
            rawText = rawText.lower()
            rawText = re.sub(r',', '', str(rawText)) # removes all commas 
            rawText = re.sub(r'\b\.\d+\b', '', str(rawText)) # removes all ints
            #print(rawText)

            newText = search(rawText)
            #print(newText)
            newInfo = list(np.append(all_info[info], newText)) # adding list adds the ',' back into the string array    
            updated_info = np.append(updated_info, newInfo)
    except:
        continue        

print(updated_info)

# split the array into equally divided arrays and convert to pandas for further processing
updated_info = np.split(updated_info, len(updated_info)/8) 
updated_info = pd.DataFrame(updated_info, columns = ['price', 'propertyAddress', 'propertyType', 'noOfBeds', 'foundFloorPlanImage', 'mapLocation', 'date', 'area'])
updated_info = updated_info.drop(columns='foundFloorPlanImage')

# Tidy data up
updated_info['noOfBeds'] = updated_info['noOfBeds'].str[1:]
date = pd.DataFrame(updated_info['date'].str.split('on', expand=True))
date.columns = ['status', 'date']
updated_info = updated_info.drop(columns='date')
updated_info = updated_info.join(date)
print(updated_info)

1 out of 803
2 out of 803
3 out of 803
4 out of 803
5 out of 803
6 out of 803
7 out of 803
8 out of 803
9 out of 803
10 out of 803
11 out of 803
12 out of 803
13 out of 803
14 out of 803
15 out of 803
16 out of 803
17 out of 803
18 out of 803
19 out of 803
20 out of 803
21 out of 803
22 out of 803
23 out of 803
24 out of 803
25 out of 803
26 out of 803
27 out of 803
28 out of 803
29 out of 803
30 out of 803
31 out of 803
32 out of 803
33 out of 803
34 out of 803
35 out of 803
36 out of 803
37 out of 803
38 out of 803
39 out of 803
40 out of 803
41 out of 803
42 out of 803
43 out of 803
44 out of 803
45 out of 803
46 out of 803
47 out of 803
48 out of 803
49 out of 803
50 out of 803
51 out of 803
52 out of 803
53 out of 803
54 out of 803
55 out of 803
56 out of 803
57 out of 803
58 out of 803
59 out of 803
60 out of 803
61 out of 803
62 out of 803
63 out of 803
64 out of 803
65 out of 803
66 out of 803
67 out of 803
68 out of 803
69 out of 803
70 out of 803
71 out of 803
72 out of 803
7

In [287]:
# Beginning of CrystalRoof 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

propertyAddress = updated_info['propertyAddress'] 

# Initialize webdriver
op = webdriver.ChromeOptions()
op.add_argument('headless')
driver = webdriver.Chrome(options=op)

allGeneralInfo = np.array([])
i = 0

# loop over all the data within propertyAddress array
for address in propertyAddress:
    start = time.time()
    # Navigate to website
    driver.get('https://crystalroof.co.uk/')

    # Find and interact with search field
    input_field = driver.find_element(By.XPATH, "//input[@role='searchbox']")

    input_field.send_keys(address, Keys.ARROW_DOWN)
    time.sleep(2)
    input_field.send_keys(Keys.RETURN)
    time.sleep(2)
    driver.get(driver.current_url)
    time.sleep(2)

    # Scrape data from website
    class_a6_a = driver.find_elements(By.XPATH, "//p[@class='a6_a']")
    class_a7_c = driver.find_elements(By.XPATH, "//*[@class='a7_c']")
    class_a3_a = driver.find_elements(By.XPATH, "//*[@class='a3_a']")
    class_a6_a_cv_b = driver.find_elements(By.XPATH, "//h2[@class='a6_a cv_b']")

    generalInfo = []
    columnNames = ['Income', 'Crime', 'Transport', '?', 'No. of Stations', 'Food Stores', '?', '?', '?', 'No. of Primary Schools', 'No. of Secondary Schools', 'Air Quality']

    for elements in class_a6_a + class_a7_c + class_a3_a + class_a6_a_cv_b:
        generalInfo.append(elements.text)

    generalInfo = [x for x in generalInfo if x] #remove empty strings
    desired_elements = 12

    if len(generalInfo) < desired_elements:
        padded_arr = np.pad(generalInfo, (0, desired_elements-len(generalInfo)), mode='constant')
    # elseif len(generalInfo) < desired_elements:
    
    else:
        padded_arr = generalInfo

    allGeneralInfo = np.append(allGeneralInfo, padded_arr)
    i = i + 1
    end = time.time()
    print(str(i)+' out of '+str(len(propertyAddress))+f' - {round(((end - start)*(len(propertyAddress)-i))/60,2)} mins remaining')
    if i == 100:
        break

generalInfo_reshaped = np.array(allGeneralInfo).reshape(-1, 12)
print(generalInfo_reshaped)
# Create dataframe
crystalRoofData = pd.DataFrame(generalInfo_reshaped, columns=columnNames)
crystalRoofData = crystalRoofData.replace(to_replace='[^0-9/]', value='', regex=True)
crystalRoofData = crystalRoofData.drop(columns=["?"])
crystalRoofData = crystalRoofData.drop(columns=["Air Quality"])
print(crystalRoofData)

driver.quit()


1 out of 701 104.53 mins remaining
2 out of 701 76.18 mins remaining
3 out of 701 76.49 mins remaining
4 out of 701 76.15 mins remaining
5 out of 701 75.62 mins remaining
6 out of 701 75.61 mins remaining
7 out of 701 74.95 mins remaining
8 out of 701 75.36 mins remaining
9 out of 701 74.46 mins remaining
10 out of 701 75.52 mins remaining
11 out of 701 75.12 mins remaining
12 out of 701 74.81 mins remaining
13 out of 701 74.55 mins remaining
14 out of 701 74.76 mins remaining
15 out of 701 74.82 mins remaining
16 out of 701 74.84 mins remaining
17 out of 701 74.52 mins remaining
18 out of 701 74.45 mins remaining
19 out of 701 74.49 mins remaining
20 out of 701 74.04 mins remaining
21 out of 701 74.36 mins remaining
22 out of 701 73.82 mins remaining
23 out of 701 73.87 mins remaining
24 out of 701 73.47 mins remaining
25 out of 701 73.07 mins remaining
26 out of 701 73.41 mins remaining
27 out of 701 73.54 mins remaining
28 out of 701 73.09 mins remaining
29 out of 701 73.16 mins rem

ValueError: cannot reshape array of size 1209 into shape (12)

In [167]:
updated_info.to_csv('output.csv', index=False) # comment out if you want to use the old data

loadData = pd.read_csv(r'output.csv')
frames = [loadData, updated_info]
combinedData = pd.concat(frames)
combinedData = combinedData.drop_duplicates(subset=['mapLocation'])

combinedData['price'] = combinedData['price'].replace('[\£,]', '', regex=True).astype(float)
combinedData['area'] = combinedData['area'].astype(float)
combinedData['priceRank'] = combinedData['price'].rank(ascending=1)
combinedData['areaRank'] = combinedData['area'].rank(ascending=0)
combinedData['Rank Sum'] = combinedData.priceRank + combinedData.areaRank
combinedData = combinedData.sort_values(by = 'Rank Sum', ascending = True)

combinedData.to_csv('output_processed.csv', index=False)
print(combinedData)

        price                       propertyAddress    propertyType noOfBeds  \
640  400000.0         Winchester Road, Edmonton, N9        Terraced        3   
531  450000.0            Annesley Avenue, Colindale  End of Terrace        4   
557  435000.0        Cambourne Avenue, Edmonton, N9        Terraced        3   
456  425000.0               Bridport Road, Edmonton        Terraced        3   
262  475000.0        Yarmouth Crescent, London, N17  End of Terrace        3   
..        ...                                   ...             ...      ...   
600  700000.0                   Abbey Road, Enfield      Town House        4   
329  700000.0  Cumbrian Gardens, Golders Green, NW2   Semi-Detached        3   
462  700000.0                Springfield Mount, NW9   Semi-Detached        4   
523  700000.0                   Hampden Way, London        Terraced        3   
700  700000.0                 Ashfield Road, London   Semi-Detached        3   

                                       